# 資管四 唐瑋廷 B05705043 HW3

In [27]:
import numpy as np
import pandas as pd
from scipy.special import expit

## Q1.1

讀取&分析data

In [28]:
remove_list = ['workclass', 'occupation', 'native-country']
train_data = pd.read_csv('adult.data', skipinitialspace=True)
for c in remove_list:
    train_data = train_data[~train_data[c].isin(['?'])]
train_data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


把 binary variable 轉成 one-hot

In [29]:
train_x = pd.DataFrame(pd.get_dummies(train_data.drop('label', axis=1)))
train_x.shape

(30162, 104)

檢查 one-hot feature，剔除樣本數不到10的 feature

In [30]:
for c in train_x.columns[6:]: # first 6 columns are continuous varibles
    if sum(train_x[c]) < 10:
        train_x = train_x.drop(c, axis=1)
features = train_x.columns # use of test data
train_x.shape

(30162, 102)

labels

In [31]:
label_dict = {'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1}
train_y = train_data['label'].map(label_dict)
train_y.shape

(30162,)

讀取 test data 並只保留 training data 有的 features

In [32]:
test_data = pd.read_csv('adult.test', skipinitialspace=True)
for c in remove_list:
    test_data = test_data[~test_data[c].isin(['?'])]
test_data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,1.896164e+05,10.112749,1120.301594,89.041899,40.951594
std,13.380676,1.056150e+05,2.558727,7703.181842,406.283245,12.062831
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.166550e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.385888e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000


In [33]:
test_x = pd.DataFrame(pd.get_dummies(test_data.drop('label', axis=1)))
test_x.shape

(15060, 103)

In [34]:
for c in test_x.columns[6:]: # first 6 columns are continuous varibles
    if c not in features:
        test_x = test_x.drop(c, axis=1)
test_x.shape

(15060, 102)

In [35]:
test_y = test_data['label'].map(label_dict)
test_y.shape

(15060,)

In [36]:
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()
train_y = train_y.to_numpy()
test_y = test_y.to_numpy()

## Q1.2

$$E(\omega) = \frac{1}{2}\omega^T\Lambda\omega - \sum_{i=1}^n\{t_nln(y_n) + (1-t_n)ln(1-y_n)\},\,y_n = \frac{1}{1+exp(-\omega^Tx_n)}$$\
$$\nabla E(\omega) = \frac{1}{2}(\Lambda^T + \Lambda)\omega + \sum_{i=1}^n (y_i - t_i)\phi_i = \Lambda\omega + \phi^T (y - t)$$\
$$H = \nabla\nabla E(\omega) = \Lambda + \sum_{i=1}^n y_i(1-y_i)\phi_i \phi_i^T = \Lambda + \phi R \phi^T$$
<center>where R is a NxN diagonal matrix with elements: $R_{nn} = y_n(1-y_n)$<\center>

## Q1.3

In [37]:
class mylogistic_l2():
    def __init__(self, reg_vec, max_iter = 1000, tol = 1e-5, add_intercept = True):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
       
    def fit(self, x, t):
        # add intercept
        if self.add_intercept:
            x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
    
        # init w
        b = self.reg_vec.trace() / self.reg_vec.shape[0]
        self.w = np.linalg.inv(x.T.dot(x) + b * np.identity(x.shape[1])).dot(x.T).dot(t)
        
        last_err = float("-inf")
        for _ in range(self.max_iter):
            # Newton-Raphson optimization method
            y = expit(self.w.T.dot(x.T))
            cur_err = self.err(y, t)
#             print(cur_err, (self.w**2).sum())
            if (cur_err - last_err) < self.tol:
                break
            last_err = cur_err
            
            gradient = self.reg_vec.dot(self.w) + x.T.dot(y-t)
            r = np.diagflat(y*(1-y))
            h = self.reg_vec + x.T.dot(r).dot(x)
            
            self.w = self.w - np.linalg.inv(h).dot(gradient)
            
            pred = np.where(y > 0.5, 1, 0)
            print(f'acc: {sum(np.equal(pred, t)) / t.shape[0]}')
    
    def predict(self, x):
        if self.add_intercept:
            x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
        p = expit(self.w.T.dot(x.T))
        return np.where(p > 0.5, 1, 0)
    
    def err(self, y, t):
        y = np.clip(y, 1e-12, 1.-1e-12)
        return np.sum(self.w.T.dot(self.reg_vec).dot(self.w))/2 - np.sum(t*np.log(y) + (1-t)*np.log(1-y)) / y.shape[0]

### case 1: lambda = 1 for all coefficients

In [38]:
logic1 = mylogistic_l2(np.identity(train_x.shape[1]+1))

In [39]:
logic1.fit(train_x, train_y)

acc: 0.45040116703136396
acc: 0.8377428552483257
acc: 0.8466282076785359
acc: 0.8493137059876666
acc: 0.8496784032889065
acc: 0.8495789403885684
acc: 0.8495789403885684
acc: 0.8495789403885684


predict & show test accuracy and learned w

In [42]:
predict = logic1.predict(test_x)
acc = sum(np.equal(predict, test_y)) / test_y.shape[0]
print(f'test accuracy: {acc}')
print(f'w: {logic1.w}')

test accuracy: 0.8480743691899071
w: [ 2.48591297e-02  7.26281835e-07  1.85806163e-01  3.16590245e-04
  6.38679434e-04  2.90215841e-02  1.89694410e-01 -4.97719502e-01
 -3.10975203e-01 -1.29181492e-01 -7.94111558e-01 -6.18745004e-01
 -1.17270706e+00 -4.52249248e-01 -5.44907404e-01 -3.82721833e-01
 -9.99810311e-02 -2.62922226e-01 -6.29791390e-01 -4.96565840e-01
 -3.08326937e-01 -1.27584811e-01  1.28534378e-01  5.87953356e-01
 -2.50481307e-01  2.99991415e-01 -1.37579993e+00  6.82097030e-01
 -1.00989638e-01 -1.00975740e+00  1.19612260e+00  8.38966951e-01
 -9.56673074e-01 -1.50523219e+00 -1.08232392e+00 -8.14848370e-01
 -8.78492153e-02 -2.46913578e-02  7.15174653e-01 -1.06504054e+00
 -7.73348089e-01 -3.53722827e-01 -9.03671285e-01 -1.69395624e+00
  4.29670534e-01  4.99488476e-01  2.05962312e-01  5.68478621e-01
 -1.78947443e-01 -5.76254831e-01 -3.75781230e-01 -1.12608336e+00
 -1.50192093e+00 -5.01524893e-01  7.47819833e-01 -1.02439618e+00
 -2.92668734e-01 -6.35195702e-01 -8.96890178e-01 -4.8

### case 2: lambda = 1 for all but the intercept, no regularization for intercept term.

In [55]:
reg_vec = np.identity(train_x.shape[1]+1)
reg_vec[-1][-1] = 0
logic2 = mylogistic_l2(reg_vec)

In [56]:
logic2.fit(train_x, train_y)

acc: 0.45040116703136396
acc: 0.8376765466481002
acc: 0.8466613619786486
acc: 0.8493137059876666
acc: 0.8495789403885684
acc: 0.8496120946886812
acc: 0.8496120946886812
acc: 0.8496120946886812


predict & show test accuracy and learned w

In [57]:
predict = logic2.predict(test_x)
acc = sum(np.equal(predict, test_y)) / test_y.shape[0]
print(f'test accuracy: {acc}')
print(f'w: {logic2.w}')

test accuracy: 0.847808764940239
w: [ 2.54336822e-02  7.50706804e-07  2.95324923e-01  3.17024478e-04
  6.39652126e-04  2.94914512e-02  7.05717318e-01  1.78026500e-02
  2.09126595e-01  3.82731145e-01 -2.79821711e-01 -1.04552949e-01
 -9.31003048e-01  9.08118762e-02 -1.06210390e-01 -5.68864880e-02
  7.06129017e-01  5.36014978e-01  1.16361327e-01  1.37527872e-01
 -4.01951496e-01 -1.13370201e-01 -7.43801938e-02  6.79669927e-02
 -1.95742294e-02 -1.15929727e-02 -1.15969699e+00  2.67049827e-01
  2.18010721e-02 -5.26538178e-01  1.61452758e+00  1.36751001e+00
 -4.92456781e-01 -1.01532722e+00 -6.05766415e-01 -3.41949002e-01
  1.64109182e-01  2.28422252e-01  9.64856017e-01 -8.17967488e-01
 -5.20782272e-01 -9.91243550e-02 -6.49283758e-01 -1.55300394e+00
  6.78427764e-01  7.51030497e-01  4.55411798e-01  8.18714335e-01
  7.31938653e-02 -4.21151206e-02  1.99456866e-01 -5.83542666e-01
 -9.36996740e-01  7.53014758e-02  1.28789619e+00 -3.72048376e-01
  3.94360616e-01  4.30693947e-02 -2.61235808e-01  1.95

### case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for intercept term.

In [61]:
reg_vec = np.diagflat(np.array([1]*6 + [0.5]*(train_x.shape[1]-6) + [0]))
logic3 = mylogistic_l2(reg_vec)

In [62]:
logic3.fit(train_x, train_y)

acc: 0.4504343213314767
acc: 0.8378091638485512
acc: 0.8468271334792122
acc: 0.8492805516875539
acc: 0.8496784032889065
acc: 0.8496784032889065
acc: 0.8496452489887939
acc: 0.8496452489887939


predict & show test accuracy and learned w

In [63]:
predict = logic3.predict(test_x)
acc = sum(np.equal(predict, test_y)) / test_y.shape[0]
print(f'test accuracy: {acc}')
print(f'w: {logic3.w}')

test accuracy: 0.847675962815405
w: [ 2.54757306e-02  7.51944312e-07  3.19092342e-01  3.17319913e-04
  6.40115460e-04  2.95135991e-02  7.66990116e-01  7.62915465e-02
  2.68338059e-01  4.43128005e-01 -2.20821080e-01 -4.63422172e-02
 -1.28758443e+00  2.18992475e-01 -2.26138731e-03  2.18816236e-02
  9.74981437e-01  7.54034100e-01  2.92224730e-01  2.93070078e-01
 -4.19612080e-01 -1.04346489e-01 -1.12963414e-01 -3.78240642e-02
  3.76517836e-02 -7.30516163e-02 -2.08394383e+00  1.86338344e-01
  5.48283061e-02 -5.72016463e-01  1.82571096e+00  1.39641689e+00
 -5.47067026e-01 -1.05936562e+00 -6.55642151e-01 -3.88036591e-01
  2.36234456e-01  3.00253533e-01  1.03825745e+00 -7.52669809e-01
 -4.53424415e-01 -2.69071693e-02 -5.82347571e-01 -2.00233647e+00
  7.51037233e-01  8.27361969e-01  5.28330008e-01  8.95057963e-01
  1.45241841e-01 -8.34438407e-02  2.32699144e-01 -5.92702337e-01
 -9.22764551e-01  1.11240624e-01  1.25497096e+00 -3.83664192e-01
  4.13058381e-01  4.13741738e-02 -2.63868161e-01  1.93

## Q1.5

In [43]:
from sklearn.linear_model import LogisticRegression

In [46]:
clf = LogisticRegression(penalty='l2', max_iter=100)
clf.fit(train_x, train_y)
clf.predict(test_x)

0.7926958831341302

In [45]:
clf.score(test_x, test_y)

0.7926958831341302

In [47]:
clf.coef_

array([[ 3.81505226e-03, -3.42208793e-06, -2.86806336e-03,
         3.35205364e-04,  7.75880361e-04, -1.82173103e-02,
         4.44679025e-04,  1.39450319e-04, -6.02054132e-03,
         1.06410737e-03, -9.03960814e-05, -8.32870382e-05,
        -1.81146254e-05, -7.09499410e-04, -1.05052212e-03,
        -3.31609852e-04, -1.24935061e-04, -2.53074786e-04,
        -4.94286894e-04, -3.98127449e-04, -9.97419202e-05,
        -1.28378645e-04,  2.77673551e-03,  6.82903077e-04,
        -4.66133137e-03,  1.75482048e-03, -5.15499570e-05,
         8.29697695e-04, -2.30520166e-03, -3.16187453e-03,
         2.18334910e-05,  1.04700807e-02, -3.02514577e-04,
        -1.01326404e-02, -8.04850238e-04, -6.54136786e-04,
        -2.33561267e-03, -8.27663339e-04,  3.20522029e-03,
        -7.31337796e-04, -1.23209320e-03, -1.21118311e-03,
        -3.31924428e-03, -1.63144337e-04,  2.46347924e-03,
         1.98375023e-04, -2.67510070e-04,  1.11803231e-04,
        -4.49302843e-04,  9.39796170e-03, -6.01084145e-0

In [281]:
tmp = test_x

In [294]:
w = np.linalg.inv(tmp.T.dot(tmp) + 1 * np.identity(tmp.shape[1])).dot(tmp.T).dot(test_y)

yy = expit(w.T.dot(tmp.T))
gradient = np.identity(tmp.shape[1]).dot(w) + tmp.T.dot(yy-test_y)
gradient.shape
r = np.diagflat(yy*(1-yy))
h = np.identity(tmp.shape[1]) + tmp.T.dot(r).dot(tmp)
new_w = w - np.linalg.inv(h).dot(gradient)
new_w.shape

(105,)

In [299]:
yy = np.clip(yy, 1e-12, 1.-1e-12)
np.sum(w.T.dot(np.identity(tmp.shape[1])).dot(w))/2 - np.sum(test_y*np.log(yy) + (1-test_y)*np.log(1-yy)) / yy.shape[0]

1.0665434373666685